# MapReduce

## ... for pedestrians

In [ ]:
def square(x):
    return x**2

In [ ]:
map(square, range(10))

In [ ]:
[x**2 for x in range(10)]

In [ ]:
def add_numbers(x, y):
    return x + y

In [ ]:
reduce(add_numbers, range(10))

In [ ]:
sum(range(10))

In [ ]:
add_numbers(0, add_numbers(1, add_numbers(2, add_numbers(3, 4))))

In [ ]:
reduce(add_numbers, range(5))

## My first MapReduce

In [ ]:
reduce(add_numbers, map(square, range(10)))

### but what about the keys?

In [ ]:
pairs = map(lambda x: (x % 2 == 0, x**2 + x), range(10))

In [ ]:
pairs

In [ ]:
from collections import defaultdict

In [ ]:
def reduceByKey(reduce_fn, iterable):
    chunks = defaultdict(list)
    for k, v in iterable:
        chunks[k].append(v)
    for k in chunks:
        chunks[k] = reduce(reduce_fn, chunks[k])
    return chunks

In [ ]:
reduceByKey(add_numbers, pairs)

## Staring Spark Locally

    PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ~/src/spark-2.1.0-bin-hadoop2.7/bin/pyspark 

## The spark context

In [ ]:
sc

## Baby steps

In [ ]:
RDD = sc.parallelize(range(10))

In [ ]:
RDD

In [ ]:
RDD.collect()

In [ ]:
RDD.count()

In [ ]:
RDD.map(lambda x: x**2).collect()

## Loading data

In [ ]:
wine = sc.textFile('data/winequality-white.csv')

In [ ]:
wine.take(5)

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
obama_url = 'https://en.wikipedia.org/wiki/Barack_Obama'
obama_soup = BeautifulSoup(requests.get(obama_url).text, 'lxml')

for unwanted in  obama_soup(['script', 'style']):
    unwanted.decompose()
obama_text = obama_soup.get_text()

obamaRDD = sc.parallelize(obama_text.split())

## Basic RDD operations

In [ ]:
obamaRDD.take(10)

In [ ]:
word_lengths = obamaRDD.map(lambda x: len(x))

In [ ]:
word_lengths.take(5)

In [ ]:
import re

In [ ]:
re.match('\w+', '') == None

In [ ]:
re.match('\w+', '-') == None

In [ ]:
re.match('\w+', 'Obama') == None

In [ ]:
clean_RDD = obamaRDD.filter(lambda x: re.match('\w+', x))

In [ ]:
clean_RDD.take(5)

In [ ]:
word_lengths = clean_RDD.map(lambda x: len(x))

In [ ]:
number_of_words = clean_RDD.count()

In [ ]:
mean_wl = word_lengths.reduce(lambda x, y: x + y) / float(number_of_words)

In [ ]:
mean_wl

In [ ]:
from operator import add

In [ ]:
word_lengths.reduce(add) / float(number_of_words)

In [ ]:
word_lengths.sum()

In [ ]:
word_counts = (
    clean_RDD
      .map(lambda x: (x, 1))
      .reduceByKey(add))

In [ ]:
word_counts.take(5)

In [ ]:
word_counts = (
    clean_RDD
      .map(lambda x: x.lower())
      .map(lambda x: (x, 1))
      .reduceByKey(add))

In [ ]:
word_counts.take(5)

In [ ]:
word_counts.takeOrdered(10, lambda (_, v): -v)

In [ ]:
class DonTryThisAtHome(object):
    def my_add(self, a, b):
        return a + b + 1
    def sketchy_top_words(self, rdd, n):
        return (rdd
               .map(lambda x: x.lower())
               .map(lambda x: (x, 1))
               .reduceByKey(self.my_add)
               .takeOrdered(5, lambda (_, v): -v))

In [ ]:
DonTryThisAtHome().sketchy_top_words(clean_RDD, 5)

In [ ]:
counter = 0
def my_count(_):
    global counter
    counter += 1

In [ ]:
my_count(123122)

In [ ]:
counter

In [ ]:
obamaRDD.foreach(my_count)

In [ ]:
counter

In [ ]:
sc.parallelize(['a', 1.2, []]).collect()

In [ ]:
acc = sc.accumulator(0)

In [ ]:
clean_RDD.foreach(lambda _: acc.add(1))

In [ ]:
acc.value

## Monitor Performance

http://localhost:4040/jobs/

## Set-like Operations

In [ ]:
a = sc.parallelize([1,2,3,4,4])
b = sc.parallelize([3,4,5])

In [ ]:
a.distinct().collect()

In [ ]:
a.union(b).collect()

In [ ]:
a.intersection(b).collect()

In [ ]:
a.cartesian(b).take(10)

In [ ]:
a.mean(), a.stdev(), a.sum()

## Other Operations of note

In [ ]:
sc.parallelize(range(100)).sample(True, 0.1).collect()

In [ ]:
a = sc.parallelize([('a', 1), ('b', 2)])
b = sc.parallelize([('c', 3), ('b', 4)])
a.join(b).collect()

In [ ]:
r = sc.parallelize([(1, 1), (1, 2), (2, 1)]).groupByKey().collect()

In [ ]:
r

In [ ]:
[(k, list(v)) for k,v in r]

## Aggregate

In [ ]:
import numpy as np

In [ ]:
def seqOp((N, partial_sum), value):
    return N + 1, partial_sum + value
def combOp((N1, partial_sum_1), (N2, partial_sum_2)):
    return N1 + N2, partial_sum_1 + partial_sum_2
values = np.random.standard_normal(1000)
N, value_sum = sc.parallelize(values).aggregate((0, 0), seqOp, combOp)

In [ ]:
mean = value_sum / float(N)

In [ ]:
mean

In [ ]:
values.mean()

# Application: Search using tf-idf

In [ ]:
def tf(term, RDD):
    tlower = term.lower()
    def seqOp((N, term_count), value):
        if value.lower() == tlower:
            return N+1, term_count+1
        else:
            return N+1, term_count
    def combOp((N1, c1), (N2, c2)):
        return N1+N2, c1+c2
    N, term_count = RDD.aggregate((0, 0), seqOp, combOp)
    return float(term_count) / N

In [ ]:
tf('and', clean_RDD)

In [ ]:
tf('a', clean_RDD)

In [ ]:
tf('obama', clean_RDD)

In [ ]:
def makeRDD(url):
    soup = BeautifulSoup(requests.get(url).text, 'lxml')
    for unwanted in soup(['script', 'style']):
        unwanted.decompose()
    text = soup.get_text()
    return sc.parallelize(text.split()).filter(lambda x: re.match('\w+', x))

In [ ]:
bushRDD = makeRDD('https://en.wikipedia.org/wiki/George_W._Bush')

In [ ]:
tf('obama', bushRDD)

In [ ]:
trumpRDD = makeRDD('https://en.wikipedia.org/wiki/Donald_Trump')

In [ ]:
def search(term):
    for RDD, name in [(clean_RDD, 'obama'), (bushRDD, 'bush'), (trumpRDD, 'trump')]:
        print name, tf(term, RDD)

In [ ]:
search('hotels')

In [ ]:
search('war')

In [ ]:
search('hawaii')